### 모델 출력 테스트

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 라이브러리 설치
!pip install -q transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.3 MB/s eta 0:00:00


In [ ]:
! git config --global credential.helper store

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `NLP_HW` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured 

In [ ]:
!hf auth whoami

user:  jyering


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

model_name = "jyering/clarifying_sft_phi"

CUDA 사용 가능: True


In [ ]:
#GPU 자원을 위해 양자화
quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/568 [00:00<?, ?B/s]

In [ ]:
#기존 chat template 확인
print(tokenizer.chat_template)

{{ '<|system|>Your name is Phi, an AI math expert developed by Microsoft.' }}{% for message in messages %}{% if message['role'] == 'system' %} {{ message['content'] }}{% if 'tools' in message and message['tools'] is not none %}{{ '<|tool|>' + message['tools'] + '<|/tool|>' }}{% endif %}{% endif %}{% endfor %}{{ '<|end|>' }}{% for message in messages %}{% if message['role'] != 'system' %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|end|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>' }}{% else %}{{ eos_token }}{% endif %}


In [ ]:
# 기본 시스템 메세지 삭
tokenizer.chat_template = ("{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
        "{{ '<|system|>' + message['content'] }}"
        "{% if 'tools' in message and message['tools'] is not none %}"
        "{{ '<|tool|>' + message['tools'] + '<|/tool|>' }}"
        "{% endif %}"
        "{% endif %}"
        "{% endfor %}"
        "{{ '<|end|>' }}"
        "{% for message in messages %}"
        "{% if message['role'] != 'system' %}"
        "{{ '<|' + message['role'] + '|>' + message['content'] + '<|end|>' }}"
        "{% endif %}"
        "{% endfor %}"
        "{% if add_generation_prompt %}"
        "{{ '<|assistant|>' }}"
        "{% else %}"
        "{{ eos_token }}"
        "{% endif %}"
        )

In [ ]:
print(tokenizer.chat_template)

{% for message in messages %}{% if message['role'] == 'system' %}{{ '<|system|>' + message['content'] }}{% if 'tools' in message and message['tools'] is not none %}{{ '<|tool|>' + message['tools'] + '<|/tool|>' }}{% endif %}{% endif %}{% endfor %}{{ '<|end|>' }}{% for message in messages %}{% if message['role'] != 'system' %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|end|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>' }}{% else %}{{ eos_token }}{% endif %}


In [ ]:
system_prompt = '''You are a chatbot that generates a concise clarifying question based on the ambiguity type when a user's query is ambiguous. You do not need to explain. just generate one clarifying question.'''

In [ ]:
system_prompt = """You are a chatbot that generates a concise clarifying question based on the ambiguity type when a user's query is ambiguous. You do not need to explain. just generate one clarifying question.

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion due to missing spatial elements
- WHAT: Query output contains confusion due to missing task-specific elements

Output format: One clarifying question"""

In [ ]:
query = '[EM|UNF] When did arnides make its last appearance in the fossil record?'

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
     {"role": "user", "content": query}
    ]

In [ ]:
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
#디코딩: Top-k sampling + temperature

questions = []
for i in range(2):
  outputs = model.generate(
      **inputs,
      max_length=1048,
      temperature=1.0,
      top_k=50,
      do_sample=True
  )
  question = tokenizer.decode(outputs[0], skip_special_tokens=True)
  questions.append(question)

In [ ]:
print(questions[0])

You are a chatbot that generates a concise clarifying question based on the ambiguity type when a user's query is ambiguous. You do not need to explain. just generate one clarifying question.

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion du

In [ ]:
print(questions[1])

You are a chatbot that generates a concise clarifying question based on the ambiguity type when a user's query is ambiguous. You do not need to explain. just generate one clarifying question.

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion du

### 데이터셋 전처리

In [ ]:
#작업 경로 지정
import os
os.chdir('/content/drive/MyDrive/woke-odds')
print(os.getcwd())

/content/drive/MyDrive/woke-odds


In [ ]:
import pandas as pd

df = pd.read_parquet('df_q_c_s.parquet')

In [ ]:
print(df.shape)
df.head()

(1601, 3)


,question,category,subclass
0,Give me a list of good coffee shops?,AO,WHOM
1,Give me some Mother's Day gift ideas.,AO,WHOM
2,Help me come up with 3 ideas for a new busines...,AO,WHAT
3,Write the first paragraph of a blog post descr...,AO,WHOM
4,Give me some tips on how to train for a marathon.,AO,WHOM


In [ ]:
query_list = df.apply(
    lambda row: f"[{row['category']}|{row['subclass']}] {row['question']}",
    axis=1
).tolist()

In [ ]:
import json

with open('clarify_string.json', 'w', encoding='utf-8') as f:
    json.dump(query_list, f, ensure_ascii=False, indent=2)

In [ ]:
print(f"총 {len(query_list)}개의 쿼리 생성됨\n")
print("=== 첫 5개 예시 ===")
for i, query in enumerate(query_list[:5], 1):
    print(f"{i}. {query}")

총 1601개의 쿼리 생성됨

=== 첫 5개 예시 ===
1. [AO|WHOM] Give me a list of good coffee shops?
2. [AO|WHOM] Give me some Mother's Day gift ideas.
3. [AO|WHAT] Help me come up with 3 ideas for a new business and how I can start it
4. [AO|WHOM] Write the first paragraph of a blog post describing the Boston
5. [AO|WHOM] Give me some tips on how to train for a marathon.


### 데이터 모델 답변 생성

In [ ]:
#쿼리 리스트 로드
import json
with open('clarify_string.json', 'r', encoding='utf-8') as f:
    query_list = json.load(f)

print(f"총 {len(query_list)}개의 쿼리")
print(query_list[:3])  # 처음 3개 확인

총 1601개의 쿼리
['[AO|WHOM] Give me a list of good coffee shops?', "[AO|WHOM] Give me some Mother's Day gift ideas.", '[AO|WHAT] Help me come up with 3 ideas for a new business and how I can start it']


In [ ]:
system_prompt = """You are a chatbot that generates a concise clarifying question based on the ambiguity type when a user's query is ambiguous. You do not need to explain. just generate one clarifying question.

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion due to missing spatial elements
- WHAT: Query output contains confusion due to missing task-specific elements

Output format: One clarifying question
"""

In [ ]:
from tqdm import tqdm
import pandas as pd

results = []

for idx, query in enumerate(tqdm(query_list)):
    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ]

        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        questions = []
        for i in range(2):
            outputs = model.generate(
                **inputs,
                max_new_tokens=1048,
                temperature=1.0,
                top_k=50,
                do_sample=True
                )

            generated_question = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            questions.append(generated_question)

        results.append({
            'query': query,
            'question_1': questions[0],
            'question_2': questions[1],
            'category': df.iloc[idx]['category'],
            'subclass': df.iloc[idx]['subclass'],
            'original_question': df.iloc[idx]['question']
        })

        # 매 100개마다 중간 결과 출력
        if (idx + 1) % 100 == 0:
            print(f"\n[{idx + 1}/{len(query_list)}] 처리 완료")

    except Exception as e:
        print(f"\n오류 발생 at index {idx}: {e}")
        results.append({
            'query': query,
            'question_1': f"ERROR: {e}",
            'question_2': f"ERROR: {e}",
            'category': df.iloc[idx]['category'],
            'subclass': df.iloc[idx]['subclass'],
            'original_question': df.iloc[idx]['question']
        })

print("\n생성 완료!")

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과 저장
results_df.to_csv('clarification_response.csv', index=False)
results_df.to_parquet('clarification_response.parquet', index=False)

print(f"\n총 {len(results_df)}개 쿼리 처리 완료")

# 결과 미리보기
print("\n=== 결과 미리보기 ===")
results_df.head()

  6%|▌         | 100/1601 [28:58<3:02:19,  7.29s/it]


[100/1601] 처리 완료


 12%|█▏        | 200/1601 [46:41<5:08:39, 13.22s/it]


[200/1601] 처리 완료


 19%|█▊        | 300/1601 [1:12:50<12:27:39, 34.48s/it]

...

100%|█████████▉| 1600/1601 [5:26:57<00:43, 43.95s/it]


[1600/1601] 처리 완료


100%|██████████| 1601/1601 [5:26:58<00:00, 12.25s/it]


생성 완료!

총 1601개 쿼리 처리 완료

=== 결과 미리보기 ===


,query,question_1,question_2,category,subclass,original_question
0,[AO|WHOM] Give me a list of good coffee shops?,>\n\n\n\nQuestion: What is the city or country...,WHOM is a location?,AO,WHOM,Give me a list of good coffee shops?
1,[AO|WHOM] Give me some Mother's Day gift ideas.,>\n\n\n\nWhen do you come in with me?,">\n\nWhat type of gift do you mean, such as pe...",AO,WHOM,Give me some Mother's Day gift ideas.
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...
3,[AO|WHOM] Write the first paragraph of a blog ...,WHAT was the name of his first dog?,END,AO,WHOM,Write the first paragraph of a blog post descr...
4,[AO|WHOM] Give me some tips on how to train fo...,I'm not sure if you're asking for the training...,"<think>\nOkay, I need to generate a clarifying...",AO,WHOM,Give me some tips on how to train for a marathon.


DPO 평가용 SFT 답변 생성

In [ ]:
import pandas as pd

df = pd.read_parquet('.parquet')

# 데이터 확인
print(f"데이터셋 크기: {len(df)}")
print(f"\n컬럼: {df.columns.tolist()}")
df.head(3)

In [ ]:
output = model.generate(
    **inputs,
    max_length=1048,
    temperature=1.0,
    top_k=50,
    do_sample=True
)
question = tokenizer.decode(output[0], skip_special_tokens=True)